In [ ]:
# Import library
import pandas as pd
import math
import matplotlib.pyplot as plt

In [ ]:
# Section 1

# Change dimension of gyroscope to radian/s

# import data
df = pd.read_excel('./gyroscope.xls')

# define sampling time, in this case is 100 Hz, or 1 / 100 or 0,01
sampling_time = 0.01

In [ ]:
# define function to change dimenion of gyrscope
# with formula :
# gyro_angle * sampling_time
gyro_angle_func = lambda coordinate: coordinate * sampling_time

In [ ]:

# perform calculation
gyro_angle_x = list(map(gyro_angle_func, df['x'].to_numpy()))
gyro_angle_y = list(map(gyro_angle_func, df['y'].to_numpy()))
gyro_angle_z = list(map(gyro_angle_func, df['z'].to_numpy()))

In [ ]:
# Section 2

# Perform complementary filter calcualtion

# Step 1 - Import accelerometer dataset

df = pd.read_excel('./acceleration.xls')

In [ ]:
# Setp 2 - Define accelerometer phi angle

# define accelerometer phi angle computation function
# with formula :
# arctan(Ay / sqrt(Ax^2 + Az^2))
phi_angle_func = lambda x,y,z: math.atan(y/ math.sqrt(math.pow(x, 2) + math.pow(z, 2)))

In [ ]:
# perform accelerometer phi angle calculation with map and functional way
accelerometer_phi_angle = list(map(phi_angle_func, df['x'].to_numpy(), 
                                                   df['y'].to_numpy(), 
                                                   df['z'].to_numpy()))

In [ ]:
# Step 3 - Perform Gait Angle calculation with complementary filter function
# formula : (gyro_angle * 0.98) + (accelerometer_phi_angle * 0.02)'
gait_angle_func = lambda gyro_angle, acc_phi_angle: (gyro_angle * 0.98) + (acc_phi_angle * 0.02)

In [ ]:
# define function to mapping plot filename
map_plot_filename = lambda filename: '_'.join(list(map(str.lower, filename.split(' '))))

In [ ]:
# perform complementary filter calculation with map and functional way
gait_angle_x = list(map(gait_angle_func, gyro_angle_x, accelerometer_phi_angle))
gait_angle_y = list(map(gait_angle_func, gyro_angle_y, accelerometer_phi_angle))
gait_angle_z = list(map(gait_angle_func, gyro_angle_z, accelerometer_phi_angle))

In [ ]:
def plot_data(gait_angle, coordinate):
    plot_name = f'Knee flexion at {coordinate}-axis'
    plt.xlabel('time (s)')
    plt.ylabel('degree/s')
    plt.title(plot_name)
    plt.plot(df['time'].to_numpy(), gait_angle)
    plt.savefig(map_plot_filename(plot_name))
    plt.show()

plot_data(gait_angle_x, 'x')
plot_data(gait_angle_y, 'y')
plot_data(gait_angle_z, 'z')
